##Importing Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

##Importing Dataset

In [2]:
# Define paths
malignant_path = "/content/Malignant"
benign_path = "/content/Benign"
image_size = 256
batch_size = 32

##Loading Dataset

In [3]:
# Load dataset
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(image_size, image_size))  # Load image
        img_array = img_to_array(img) / 255.0  # Normalize
        images.append(img_array)
        labels.append(label)  # Assign label
    return images, labels

In [4]:
# Load both categories
malignant_images, malignant_labels = load_images_from_folder(malignant_path, label=1)
benign_images, benign_labels = load_images_from_folder(benign_path, label=0)

In [5]:
# Combine and shuffle data
X = np.array(malignant_images + benign_images)
y = np.array(malignant_labels + benign_labels)

##Splitting the Dataset

In [6]:
# Split dataset (80% Train, 20% Validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [7]:
# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

## Convolutional Neural Network Model

In [8]:
def create_custom_model(size=256, dropout_rate=0.5, classes=1):
    model = Sequential()
    # Block 1
    model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001), input_shape=(size, size, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))  # Early dropout

    # Block 2
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))

    # Block 3
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.4))

    # Block 4
    model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.4))

    # Block 5
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.5))  # Strongest dropout here

    # Global Average Pooling for feature reduction
    model.add(GlobalAveragePooling2D())

    # Fully Connected Layers
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    # Output Layer (Binary classification - Malignant vs. Benign)
    model.add(Dense(classes, activation='sigmoid'))  # Sigmoid for binary classification

    return model

In [9]:
# Initialize Model
model = create_custom_model(size=image_size)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

##Model Training

In [11]:
# Train Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 75s 4s/step - accuracy: 0.5400 - loss: 2.3729 - val_accuracy: 0.3797 - val_loss: 2.1953
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.5846 - loss: 2.2120 - val_accuracy: 0.3797 - val_loss: 2.2638
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.6790 - loss: 2.0943 - val_accuracy: 0.3797 - val_loss: 2.3824
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 224ms/step - accuracy: 0.7366 - loss: 2.0412 - val_accuracy: 0.3797 - val_loss: 2.5130
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 221ms/step - accuracy: 0.6942 - loss: 2.0599 - val_accuracy: 0.3797 - val_loss: 2.7166
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 0.6993 - loss: 2.0698 - val_accuracy: 0.3797 - val_loss: 2.8946
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.7495 - loss: 2.0280 - val_accuracy: 0.3797 - val_loss: 3.1169
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 0.7051 - loss: 2.1109 - val_accuracy: 0.37

In [12]:
# Evaluate Model on Validation Data
val_preds = model.predict(val_dataset)
val_preds = (val_preds > 0.5).astype(int)  # Convert probabilities to binary class

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 580ms/step


##Generate Report

In [13]:
# Generate Classification Report
report = classification_report(y_val, val_preds, target_names=["Benign", "Malignant"])
print(report)

              precision    recall  f1-score   support

      Benign       0.00      0.00      0.00        49
   Malignant       0.38      1.00      0.55        30

    accuracy                           0.38        79
   macro avg       0.19      0.50      0.28        79
weighted avg       0.14      0.38      0.21        79



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
